In [ ]:
# Setup
from IPython.display import Markdown
import jupyter_black
import pandas as pd

jupyter_black.load()


from supreme_court_predictions.models.logistic_regression import (
    LogisticRegression,
)
from supreme_court_predictions.models.random_forest import RandomForest
from supreme_court_predictions.models.xg_boost import XGBoost

# Setup for display
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

%matplotlib inline


# Loading the dataframes
from supreme_court_predictions.util.functions import get_full_data_pathway

data_path = get_full_data_pathway("clean_convokit/")
utterances = pd.read_csv(data_path + "utterances_df.csv")
advocates = pd.read_csv(data_path + "advocates_df.csv")
cases = pd.read_csv(data_path + "cases_df.csv")
conversations = pd.read_csv(data_path + "conversations_df.csv")
speakers = pd.read_csv(data_path + "speakers_df.csv")
voters = pd.read_csv(data_path + "voters_df.csv")

## Machine Learning Final Presentation
### Predicting Outcomes of US Supreme Court Oral Arguments
**Team Members: Federico Dominguez, Chanteria Milner, Jessup Jong, Michael Plunkett**

## Background
[insert]

## Datasets

### Case Information

In [ ]:
Markdown(f"""

## {greetings} {name}!

You can mix Markdown with variables thanks to `IPython.display.Markdown()`.

""")

In [15]:
cases.head(2)

,id,year,citation,title,petitioner,respondent,docket_no,court,decided_date,win_side,is_eq_divided
0,2014_13-553,2014,575 US _,Alabama Department of Revenue v. CSX Transport...,"Alabama Department of Revenue, et al.","CSX Transportation, Inc.",13-553,Roberts Court,"Mar 4, 2015",1,False
1,2014_13-895,2014,575 US _,Alabama Legislative Black Caucus v. Alabama,"Alabama Democratic Conference, et al.","Alabama, et al.",13-895,Roberts Court,"Mar 25, 2015",1,False


In [9]:
speakers.head(2)

,speaker_name,speaker_type,speaker_role,speaker_key
0,Earl Warren,J,justice,earl_warren
1,Harry F. Murphy,A,NaN,harry_f_murphy


In [10]:
voters.head(2)

,id,case_id,voter,vote
0,23291,2014_13-553,j__john_g_roberts_jr,1
1,23291,2014_13-553,j__antonin_scalia,1


In [11]:
advocates.head(2)

,id,case_id,advocate,side,role
0,23291,2014_13-553,andrew_l_brasher,1,"Solicitor General, Alabama, for the petitioner"
1,23291,2014_13-553,carter_g_phillips,0,for the respondent


In [12]:
conversations.head(2)

,id,case_id,winning_side
0,23291,2014_13-553,1
1,23252,2014_13-895,1


In [13]:
utterances.head(2)

,case_id,speaker,speaker_type,conversation_id,id,text,tokens
0,2014_13-553,j__john_g_roberts_jr,J,23291,23291__0_000,"we'll hear argument next in case no. 13553, th...","['hear', 'argument', 'case', 'alabama', 'depar..."
1,2014_13-553,andrew_l_brasher,A,23291,23291__0_001,"thank you, mr. chief justice, and may it pleas...","['thank', 'mr', 'chief', 'justice', 'court', '..."
